In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost", 
    database="solana",
    user="solana"
)

cur = conn.cursor()

# %%
cmd = """
select * from account
union 
select * from account_audit
order by slot, write_version asc
"""

import pandas as pd 
df = pd.read_sql_query(cmd, conn)
print("# of examples found:", len(df))

cur.close()
conn.close()

df.head()

# of examples found: 281


/Users/brennan/miniconda3/envs/10/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,pubkey,owner,lamports,slot,executable,rent_epoch,data,write_version,updated_on,txn_signature
0,"[b'\xdf', b'\xac', b'+', b'1', b'\xe1', b'r', ...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",7906560,10,False,0,"[b'\xd8', b'\x92', b'k', b'^', b'h', b'K', b'\...",202,2022-10-17 16:34:16.602273,"[b'a', b'\xce', b'\xa7', b'\xc8', b'\xba', b'\..."
1,"[b'\xdf', b'\xac', b'+', b'1', b'\xe1', b'r', ...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",7906560,11,False,0,"[b'\xd8', b'\x92', b'k', b'^', b'h', b'K', b'\...",213,2022-10-17 16:34:17.113816,"[b'8', b'\x0f', b'\x8d', b'.', b'\x8f', b'\xbb..."
2,"[b'\xee', b'\x1b', b']', b'\xa4', b'\xbc', b'a...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",6514560,11,False,0,"[b'd', b'\xb1', b'\x08', b'k', b'\xa8', b'A', ...",214,2022-10-17 16:34:17.113830,"[b'8', b'\x0f', b'\x8d', b'.', b'\x8f', b'\xbb..."
3,"[b'\xa2', b'\x1e', b'\xc9', b'\xd6', b'\x84', ...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",10078080,13,False,0,"[b'\n', b'\xdf', b'\x0c', b',', b'k', b'\xf5',...",232,2022-10-17 16:34:18.141927,"[b'\xe5', b'\xc2', b'p', b'\x1a', b'\xec', b'N..."
4,"[b'\xdf', b'\xac', b'+', b'1', b'\xe1', b'r', ...","[b'\xb9', b'\\', b'*', b'\x95', b'\xfc', b'\xb...",7906560,13,False,0,"[b'\xd8', b'\x92', b'k', b'^', b'h', b'K', b'\...",233,2022-10-17 16:34:18.141928,"[b'\xe5', b'\xc2', b'p', b'\x1a', b'\xec', b'N..."


In [2]:
from solana.publickey import PublicKey

datas = []
pks = []
for i, row in df.iterrows():
    pk = row.pubkey
    pk = PublicKey(b''.join([pk[i] for i in range(pk.nbytes)]))
    pks.append(str(pk))

    data = row.data
    data = b''.join([data[i] for i in range(data.nbytes)])
    datas.append(data)

df['_data'] = datas
df['pk'] = pks

In [3]:
from anchorpy import Program
from solana.publickey import PublicKey
from pathlib import Path 
from anchorpy import Idl 
import json 

file = Path('../driftpy/protocol-v2/target/idl/clearing_house.json')
with file.open() as f:
    idl_dict = json.load(f)
idl = Idl.from_json(idl_dict)
program = Program(idl, PublicKey(0), None)

import re 
with open(f'../driftpy/protocol-v2/programs/clearing_house/src/lib.rs', 'r') as f:
    data = f.read()
re_result = re.search('\[cfg\(not\(feature = \"mainnet-beta\"\)\)\]\ndeclare_id!\(\"(.*)\"\)', data)
program_id = PublicKey(re_result.group(1))
print('program pk:', program_id)

program pk: DUZwKJKAk2C9S88BYvQzck1M1i5hySQjxB4zW6tJ29Nw


In [6]:
import sys
sys.path.insert(0, '../driftpy/src/')

import json
from driftpy.addresses import * 

trial_outpath = 'tmp3'
with open(f"../backtest/{trial_outpath}/trial_no_oracle_guards/users.json", 'r') as f:
    users = json.load(f)

user_auths = [PublicKey(u) for u in users.values()]
users = [get_user_account_public_key(program_id, auth) for auth in user_auths]
user_stats = [
    get_user_stats_account_public_key(program_id, pk)
    for pk in user_auths
]

account_pks = {
    'State': get_state_public_key(program_id),
    'PerpMarket': get_perp_market_public_key(program_id, 0), # only single market rn
    'User': users,
    'UserStats': user_stats,
}
len(users)

21

In [7]:
save_path = Path('data')/trial_outpath
save_path.mkdir(exist_ok=True, parents=True)
print(save_path.absolute())

def extract(pk, type):
    type_rows = df[df['pk'] == str(pk)]
    type_history = []
    for i, r in type_rows.iterrows():
        data = r['_data']
        type_obj = program.account[type]._coder.accounts.parse(data).data
        type_history.append(type_obj)
    return type_history

i = 0
perp_market_history = extract(account_pks['PerpMarket'], 'PerpMarket')
perp_market_df = []
for perp_market in perp_market_history:
    d = perp_market.__dict__
    # todo do things
    perp_market_df.append(d)
perp_market_df = pd.DataFrame(perp_market_df)
perp_market_df.to_csv(save_path/f"perp_market_{i}.csv", index=False)

i = 0
state_history = extract(account_pks['State'], 'State')
state_df = []
for state in state_history:
    d = state.__dict__
    # todo do things
    state_df.append(d)
state_df = pd.DataFrame(state_df)
state_df.to_csv(save_path/f"state_{i}.csv", index=False)

for i, (user, user_stat) in enumerate(zip(users, user_stats)):
    user_stats_history = extract(user_stat, "UserStats")
    user_history = extract(user, "User")

    user_df = []
    for user in user_history:
        d = user.__dict__
        # todo do things
        user_df.append(d)
    user_df = pd.DataFrame(user_df)
    user_df.to_csv(save_path/f"user_{i}.csv", index=False)

    user_stats_df = []
    for user_s in user_stats_history:
        d = user_s.__dict__
        # todo do things
        user_stats_df.append(d)
    user_stats_df = pd.DataFrame(user_stats_df)
    user_stats_df.to_csv(save_path/f"user_stats_{i}.csv", index=False)

/Users/brennan/Documents/drift/py-drift/drift-sim/solana-accountsdb-plugin-postgres/data/tmp3


In [8]:
len(users), len(user_stats)

(21, 21)